# Post Hoc Explaination on Adult

# Setup

In the following cells we are going to configure our notebook. First, we clone the xai library (Naretto, Bodria) and install the necessary packages with pip. Then we import the libraries that we need

In [1]:
!git clone https://github.com/pellungrobe/XAI_course_2024

Cloning into 'XAI_course_2024'...
error: RPC failed; curl 92 HTTP/2 stream 5 was not closed cleanly before end of the underlying stream
error: 4870 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [3]:
#cd XAI_course_2024

/Users/roberto/Documents/Insegnamento/XAI/2024/XAI_course_2024


Remember to use the .yml file with the configuration:  conda config --set channel_priority flexible

In [3]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

import lime
import shap
import xgboost as xgb

from xailib.data_loaders.dataframe_loader import prepare_dataframe

from xailib.explainers.lime_explainer import LimeXAITabularExplainer
from xailib.explainers.lore_explainer import LoreTabularExplainer
from xailib.explainers.shap_explainer_tab import ShapXAITabularExplainer

from xailib.models.sklearn_classifier_wrapper import sklearn_classifier_wrapper

ModuleNotFoundError: No module named 'pandas'

# Input Data Preparation


We start by reading our data. 

We are going to work on the adult dataset. It is a classic, benchmark tabular dataset, designed for a classification task.

The Adult dataset classifies people as high ($\geq$ 50k) or low ($\leq$50k) income.

We read our data from a .csv file into a `pandas` `DataFrame`.

In [7]:
filename = 'tabular datasets/adult_clean.csv'
adult = pd.read_csv(filename, skipinitialspace=True, na_values='?', keep_default_na=True)
adult.head()

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,0,77516,13,7,14,5,1,0,2174,0,40,41,0
1,50,1,83311,13,1,1,1,1,0,0,0,13,41,0
2,38,2,215646,9,2,2,5,1,0,0,0,40,41,0
3,53,2,234721,7,1,2,1,2,0,0,0,40,41,0
4,28,2,338409,13,1,3,2,2,1,0,0,40,1,0


`class` is our target variable, the one that we want to predict

In [8]:
target = 'class'

We can look at some summary to have an idea of how our data is distributed

In [9]:
adult.describe()

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
count,30162.000000,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,2.109343,1.897938e+05,10.121312,3.332272,6.342749,2.775247,1.211823,0.324315,1092.007858,88.372489,40.931238,38.570420,0.248922
std,13.134665,0.934785,1.056530e+05,2.549995,2.708290,4.127163,1.676177,0.612461,0.468126,7406.346497,404.298370,11.979984,8.429816,0.432396
min,17.000000,0.000000,1.376900e+04,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,28.000000,2.000000,1.176272e+05,9.000000,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,40.000000,41.000000,0.000000
50%,37.000000,2.000000,1.784250e+05,10.000000,2.000000,5.000000,3.000000,1.000000,0.000000,0.000000,0.000000,40.000000,41.000000,0.000000
75%,47.000000,2.000000,2.376285e+05,13.000000,7.000000,10.000000,5.000000,1.000000,1.000000,0.000000,0.000000,45.000000,41.000000,0.000000
max,90.000000,6.000000,1.484705e+06,16.000000,7.000000,14.000000,5.000000,5.000000,1.000000,99999.000000,4356.000000,99.000000,41.000000,1.000000


In [10]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             30162 non-null  int64
 1   workclass       30162 non-null  int64
 2   fnlwgt          30162 non-null  int64
 3   education-num   30162 non-null  int64
 4   marital-status  30162 non-null  int64
 5   occupation      30162 non-null  int64
 6   relationship    30162 non-null  int64
 7   race            30162 non-null  int64
 8   sex             30162 non-null  int64
 9   capital-gain    30162 non-null  int64
 10  capital-loss    30162 non-null  int64
 11  hours-per-week  30162 non-null  int64
 12  native-country  30162 non-null  int64
 13  class           30162 non-null  int64
dtypes: int64(14)
memory usage: 3.2 MB


After the data is loaded in memory, we need to extract metadata information to automatically handle the content withint the table.

We can do this on our own, but the loader method `prepare_dataframe` can make a lot of work for us.
The method `prepare_dataframe` scans the table and extract the following information:

- `df`: is a trasformed version of the original dataframe, where discrete attributes are transformed into numerical attributes by using one hot encoding strategy;
- `feature_names`: is a list containint the names of the features after the transformation;
- `class_values`: the list of all the possible values for the class_field column;
- `numeric_columns`: a list of the original features that contain numeric (i.e. continuous) values;
- `rdf`: the original dataframe, before the transformation;
- `real_feature_names`: the list of the features of the dataframe before the transformation;
- `features_map`: it is a dictionary pointing each feature to the original one before the transformation.

In [ ]:
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = prepare_dataframe(adult, target)

# Let's train a black box on our data

Now that the data is ready we can train a classifier on it. Let's try the classic `random forest classifier`.


First, we split train and test

In [ ]:
test_size = 0.3
random_state = 42
X_train, X_test, Y_train, Y_test = train_test_split(df[feature_names], df[target],
                                                        test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=df[target])

We can then create our black box and train it. 
We should do some hyper parameter search first, possibly with cross validation, but this is just an example

In [ ]:
bb = RandomForestClassifier(n_estimators=20, random_state=random_state)
bb.fit(X_train.values, Y_train.values)
 

In [ ]:
Y_pred = bb.predict(X_train)
print(classification_report(Y_train, Y_pred))

In [ ]:
Y_pred = bb.predict(X_test)
print(classification_report(Y_test, Y_pred))

Obviously, train and test present very different metrics. 
Now we can apply a post-hoc explanation method to our black box. 

I will also train an alternative model, an xgboost classifier. The reason will become clear shortly.

In [ ]:
xgbclf = xgb.XGBClassifier().fit(X_train, Y_train)

In [ ]:
sklearn import metrics

import lime
import shap
import xgboost

# SHAP


SHAP uses game theory and shapley values to provide an explanation.

The general working of SHAP in the classical Kernel mode (default mode) is:

- Sample coalitions $z_k'\in\{0,1\}^M,\quad{}k\in\{1,\ldots,K\}$ (1 = feature present in coalition, 0 = feature absent).
- Get prediction for each $z_k'$ by first converting $z_k'$ to the original feature space and then applying model $\hat{f}: \hat{f}(h_x(z_k'))$
- Compute the weight for each $z_k'$ with the SHAP kernel.
- Fit weighted linear model.
- Return Shapley values $\phi_k$, the coefficients from the linear model.

SHAP provides several explanators, optimized for different objectives, and several visualization options.

- Kernel Explainer, it is the *real* agnostic method for computing Shap values. Hence, it works for every kind of ML model (quite slow).
- Linear Explainer, for explaining linear models. It is an exact method. It allows for an analysis with independent variables, as well as for correlated ones (you have to estimates the variables covariance matrix).
- Tree Explainer, for explaining tree-based models (tree and ensambles). Is is an exact algorithm. (Difference: conditional expectation $E_{X_j|X_{-j}}(\hat{f}(x)|x_j)$)
- Gradient Explainer, for explaining deep learning models. It is based on Integrated Gradient and it is an approximation algorithm.
- Deep Explainer, for explaining deep learning models. It is based on DEEPLift and the method computes an approximation algorithm (faster than Gradient Explainer).

For our random forest we can use the tree explainer

In [ ]:
shap_tree_exp = shap.TreeExplainer(bb)
shap_values_tree = shap_tree_exp.shap_values(X_test[0:200])

In [ ]:
shap_values_tree

The summary plot gives us an overall view of the shapley values for a bunch of records

In [ ]:
shap.summary_plot(shap_values_tree, X_test[0:200])

In [ ]:
tree_explanation_object = shap_tree_exp(X_test[0:200])

In [ ]:
tree_explanation_object

In [ ]:
shap.plots.bar(tree_explanation_object[:,:,0])

In [ ]:
shap.summary_plot(shap_values_tree[0], X_test[0:200], plot_type='bar')

In [ ]:
xgb_shap_tree_exp = shap.TreeExplainer(xgbclf)
xgb_shap_values_tree = xgb_shap_tree_exp.shap_values(X_test[0:200])

In [ ]:
xgb_shap_values_tree

In [ ]:
shap.summary_plot(xgb_shap_values_tree, X_test[0:200])

In [ ]:
shap.summary_plot(shap_values_tree[0], X_test[0:200])

We can use dependence plot to focus on one variable.

- Each dot is a single prediction (row) from the dataset.
- The x-axis is the value of the feature (from the X matrix).
- The y-axis is the SHAP value for that feature, which represents how much knowing that feature’s value changes the output of the model for that sample’s prediction.
- The color corresponds to a second feature that may have an interaction effect with the feature we are plotting (by default this second feature is chosen automatically). If an interaction effect is present between this other feature and the feature we are plotting it will show up as a distinct vertical pattern of coloring.

In [ ]:
shap.dependence_plot("age", shap_values_tree[0], X_test[0:200])

We can explore the interactions between variables.
SHAP interaction values are a generalization of SHAP values to higher order interactions.
A summary plot of a SHAP interaction value matrix plots a matrix of summary plots with the main effects on the diagonal and the interaction effects off the diagonal.

In [ ]:
shap_interaction_values_tree = shap_tree_exp.shap_interaction_values(X_test.iloc[0:20])
shap.summary_plot(shap_interaction_values_tree[1], X_test.iloc[0:20])

Running a dependence plot on the SHAP interaction values a allows us to separately observe the main effects and the interaction effects.

In [ ]:
shap.dependence_plot(
    ("age", "workclass"),
    shap_interaction_values_tree[0], X_test.iloc[0:20]
)


SHAP also provides some nice interactive visualizations using javascript

In [ ]:
shap.initjs()
shap.force_plot(shap_tree_exp.expected_value[0], shap_values_tree[0], X_test[0:200])

For now we've explored our model on a global level. Let's now look at local explanation using SHAP.

In [ ]:
shap.initjs()
shap.force_plot(shap_tree_exp.expected_value[0], shap_values_tree[0][0,:], feature_names)

In [ ]:
shap.plots.waterfall(tree_explanation_object[:,:,0][0])

In [ ]:
shap.plots._waterfall.waterfall_legacy(shap_tree_exp.expected_value[0], shap_values_tree[0][0,:], feature_names=feature_names)

Another example

Let's try a different explainer: before we used the TreeExplainer, but there are other explainers available in SHAP. Since we are dealing with a RandomForest, another possibility could be the KernelExplainer, that is completley agnostic.

KernelExplainer requires in input the predict_proba of the black-box we want to explain and a sample of the data. In this simple example we use just the first 20 records of the test set, but in principle you can pass whatever representation of your data, such as the centroids obtained from a clustering algorithm.

In [ ]:
shap_kernel_exp = shap.KernelExplainer(bb.predict_proba, X_test[0:20])
shap_values_kernel = shap_kernel_exp.shap_values(X_test[0:200])

Let's now see how the explanation change

In [ ]:
shap.initjs()
shap.force_plot(shap_kernel_exp.expected_value[0], shap_values_kernel[0], X_test[0:200])

In [ ]:
shap.summary_plot(shap_values_kernel, X_test[0:200])

In [ ]:
shap.initjs()
shap.force_plot(shap_kernel_exp.expected_value[1], shap_values_kernel[1][1,:], feature_names)

In [ ]:
shap.plots._waterfall.waterfall_legacy(shap_kernel_exp.expected_value[1], shap_values_kernel[1][1,:])

# LIME

LIME focuses on local explanations. The idea is: it generates a set of neighbours, on them it trains a linear model. 

- Select your instance of interest for which you want to have an explanation of its black box prediction.
- Perturb your dataset and  get the black box predictions for these new points.
- Weight the new samples according to their proximity to the instance of interest.
- Train a weighted, interpretable model on the dataset with the variations.
- Explain the prediction by interpreting the local model.


There are 2 kinds of neighbourhood generation:

- Gaussian
- Lhs (Latin Hypercube Sampling)

We first initialize the lime explainer on our train data, providing the names of our features

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names = feature_names)

In [ ]:
inst = X_test.values[0]

Then we can explain each data point locally
We can see each explanation from the "perspective" of the class that we want to focus on

In [ ]:
exp = explainer.explain_instance(inst, bb.predict_proba)
exp.show_in_notebook(show_table=True, show_all=False)

In [ ]:
exp = explainer.explain_instance(inst, bb.predict_proba, labels=[0])
exp.show_in_notebook(show_table=True, show_all=False)

We can also "cut" our explanation and modify the visualization

In [ ]:
exp = explainer.explain_instance(inst, bb.predict_proba, num_features=5)
exp.show_in_notebook(show_table=True, show_all=False)

You can also manage other options:

- top_labels: if not None, ignore labels and produce explanations for
                the K labels with highest prediction probabilities, where K is
                this parameter.
- num_features: maximum number of features present in explanation
            num_samples: size of the neighborhood to learn the linear model
- distance_metric: the distance metric to use for weights.
- model_regressor: sklearn regressor to use in explanation. Defaults
                to Ridge regression in LimeBase. Must have model_regressor.coef_
                and 'sample_weight' as a parameter to model_regressor.fit()
- sampling_method: Method to sample synthetic data. Defaults to Gaussian
                sampling. Can also use Latin Hypercube Sampling.

We can provide the explanation as a pyplot figure

In [ ]:
exp.as_pyplot_figure(label=0)

Or manipulate the values as a list

In [ ]:
exp.as_list(0)

In [ ]:
exp.as_map()

# Lore

Lore is an algorithm developed internally to the KDDLab. To use it we will use the xai lib that we imported at the beginning. Note that shap and lime are wrappend in the xai lib, so if you want you can replicate the previous part using the xai lib.

We then use the `sklearn_classifier_wrapper` from xai lib to wrap our black box and make it ready to explain

In [ ]:
bbox = sklearn_classifier_wrapper(bb) 

Now e could also use the imlpementations of SHAP and Lime that are present in the XAI-lib

In [ ]:
explainer = LoreTabularExplainer(bbox)

In [ ]:
inst = X_train.iloc[147].values

Lore uses various type of neighborhood generation to provide a local explanation based on factuals and counterfactuals

In [ ]:
explainer = LoreTabularExplainer(bbox)
config = {'neigh_type':'rndgen', 'size':1000, 'ocr':0.1, 'ngen':10}
explainer.fit(df, target, config)
exp = explainer.explain(inst)
print(exp)


In [ ]:
print('Instance ',inst)
print('True class ',Y_train.iloc[147])
print('Predicted class ',bb.predict(inst.reshape(1, -1)))

In [ ]:
exp.plotRules()

In [ ]:
exp.plotCounterfactualRules()